In [54]:
#문제2. 농작물 분류 범주 3개 (다중 분류) Macro F1-Score
#독립변수 X는 비료 종류, 재배월 등
#종속변수 Y는 농작물 상태(0:정상, 1:불량, 2:폐기)
#데이터 셋:불균형 데이터(정상인 Class가 대부분이고 불량, 폐기는 소수의 Class)
#결측치가 없다. 불균형 데이터셋은 데이터 분리시 stratify 옵션이 필수이다.
import numpy as np
import pandas as pd

import numpy as np
import pandas as pd

np.random.seed(42)

n_samples = 1000

#독립변수 생성
month=np.random.randint(1,13, size=n_samples)
regions=["Region_1","Region_2","Region_3","Region_4"] #재배한 월 1~12 사이의 정수
region=np.random.choice(regions, size=n_samples) #4개의 지역 중 선택 
fertilizers=["A","B","C"]
fertilizer = np.random.choice(fertilizers, size=n_samples) #비료 종류 선택
soil_moisture=np.random.uniform(10,100,size=n_samples)#토양 수분(10~100)
temperature=np.random.uniform(15,35,size=n_samples)#온도(15~35)
rainfall=np.random.uniform(0,200,size=n_samples)#강수량(0,200))
humidity=np.random.uniform(20,100,size=n_samples)#습도(20~100%)
ph_level=np.random.uniform(4.0,9.0,size=n_samples)#토양(4.0~9.0)

#종속변수 생성
condition=np.random.choice([0,1,2],size=n_samples,p=[0.7,0.2,0.1])

#데이터프레임 생성
data=pd.DataFrame({
    "Month":month,
    "Region":region,
    "Fertilizer":fertilizer,
    "soil_moisture":soil_moisture,
    "Temperature":temperature,
    "Rainfall":rainfall,
    "Humidity":humidity,
    "ph_level":ph_level,
    "Condition":condition
})
#데이터 섞은 후 700개는 train, 300개는 test로 분할
data_shuffled = data.sample(frac=1, random_state=42)
train=data_shuffled.iloc[:700]
test=data_shuffled.iloc[700:]
y_test=test['Condition'] #실제 시험에서 주어지지 않는다.
test=test.drop(['Condition'], axis=1)
train.head(3)



,Month,Region,Fertilizer,soil_moisture,Temperature,Rainfall,Humidity,ph_level,Condition
521,10,Region_4,C,53.740044,20.281240,104.680274,75.597611,6.349022,1
737,4,Region_1,B,68.303570,24.851381,129.120515,55.130453,6.844607,1
740,10,Region_2,B,55.277556,22.228076,156.378183,86.111225,7.697816,0


In [55]:
#분류 모델을 사용하여 macro F1 Score를 산출하시오.
#예측값을 pred 변수에 담아 csv파일에 담아 제출하시오.
import numpy as np
import pandas as pd 

x_train = train.drop(['Condition'],axis=1)
y_train = train['Condition']
x_test =test

In [56]:
#데이터의 행과 열 확인
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)

(700, 8)
(300, 8)
(700,)


In [57]:
#초기 데이터 확인
print(x_train.head(3))
print(x_test.head(3))
print(y_train.head(3))

     Month    Region Fertilizer  soil_moisture  Temperature    Rainfall  \
521     10  Region_4          C      53.740044    20.281240  104.680274   
737      4  Region_1          B      68.303570    24.851381  129.120515   
740     10  Region_2          B      55.277556    22.228076  156.378183   

      Humidity  ph_level  
521  75.597611  6.349022  
737  55.130453  6.844607  
740  86.111225  7.697816  
     Month    Region Fertilizer  soil_moisture  Temperature    Rainfall  \
788      5  Region_4          A      66.944069    30.005315   23.990588   
640     11  Region_4          C      94.963788    20.540250  110.694355   
889      8  Region_2          A      52.642885    34.361528   13.520950   

      Humidity  ph_level  
788  80.531724  6.060089  
640  55.706109  7.112877  
889  48.325165  5.118013  
521    1
737    1
740    0
Name: Condition, dtype: int32


In [58]:
#변수명과 데이터 타입이 매칭이 되는지 결측치가 있는지 확인해보세요.

print(x_train.info())
print(x_test.info())
print(y_train.info())

<class 'pandas.core.frame.DataFrame'>
Index: 700 entries, 521 to 150
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Month          700 non-null    int32  
 1   Region         700 non-null    object 
 2   Fertilizer     700 non-null    object 
 3   soil_moisture  700 non-null    float64
 4   Temperature    700 non-null    float64
 5   Rainfall       700 non-null    float64
 6   Humidity       700 non-null    float64
 7   ph_level       700 non-null    float64
dtypes: float64(5), int32(1), object(2)
memory usage: 46.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 300 entries, 788 to 102
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Month          300 non-null    int32  
 1   Region         300 non-null    object 
 2   Fertilizer     300 non-null    object 
 3   soil_moisture  300 non-null    float64
 4   Temperature    300 no

In [59]:
#x_train과 x_test 데이터 기초통계량을 잘 비교하세요.

print(x_train.describe())
print(x_test.describe())
print(y_train.describe())

            Month  soil_moisture  Temperature    Rainfall    Humidity  \
count  700.000000     700.000000   700.000000  700.000000  700.000000   
mean     6.302857      54.809396    24.982946   97.327036   59.383117   
std      3.511639      26.180474     5.719321   57.836677   23.185064   
min      1.000000      10.012122    15.000233    0.006144   20.075473   
25%      3.000000      33.324074    20.064593   46.727760   39.214022   
50%      6.000000      53.888270    25.041492   95.701944   58.721330   
75%      9.000000      77.927692    29.967260  145.737345   79.188142   
max     12.000000      99.803877    34.991154  199.549878   99.948024   

         ph_level  
count  700.000000  
mean     6.454647  
std      1.453100  
min      4.007142  
25%      5.178267  
50%      6.469068  
75%      7.737086  
max      8.997303  
            Month  soil_moisture  Temperature    Rainfall    Humidity  \
count  300.000000     300.000000   300.000000  300.000000  300.000000   
mean     6.80666

In [60]:
print(x_train.describe(include='object'))
print(x_test.describe(include='object'))

          Region Fertilizer
count        700        700
unique         4          3
top     Region_1          B
freq         184        249
          Region Fertilizer
count        300        300
unique         4          3
top     Region_2          A
freq          87        110


In [61]:
#데이터 전처리 및 분리
print(x_train.isnull().sum())
print(x_test.isnull().sum())
print(y_train.isnull().sum())

Month            0
Region           0
Fertilizer       0
soil_moisture    0
Temperature      0
Rainfall         0
Humidity         0
ph_level         0
dtype: int64
Month            0
Region           0
Fertilizer       0
soil_moisture    0
Temperature      0
Rainfall         0
Humidity         0
ph_level         0
dtype: int64
0


In [62]:
#뒤에 레이블인코딩 보여주기
x_train2 = x_train.copy()
x_test2=x_test.copy()
y_train2=y_train.copy()

#원핫인코딩(가변수처리)
x_train=pd.get_dummies(x_train)
x_test=pd.get_dummies(x_test)
print(x_train.info())
print(x_test.info())


<class 'pandas.core.frame.DataFrame'>
Index: 700 entries, 521 to 150
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Month            700 non-null    int32  
 1   soil_moisture    700 non-null    float64
 2   Temperature      700 non-null    float64
 3   Rainfall         700 non-null    float64
 4   Humidity         700 non-null    float64
 5   ph_level         700 non-null    float64
 6   Region_Region_1  700 non-null    bool   
 7   Region_Region_2  700 non-null    bool   
 8   Region_Region_3  700 non-null    bool   
 9   Region_Region_4  700 non-null    bool   
 10  Fertilizer_A     700 non-null    bool   
 11  Fertilizer_B     700 non-null    bool   
 12  Fertilizer_C     700 non-null    bool   
dtypes: bool(7), float64(5), int32(1)
memory usage: 40.3 KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 300 entries, 788 to 102
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dty

In [63]:
print(x_train2.info())

<class 'pandas.core.frame.DataFrame'>
Index: 700 entries, 521 to 150
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Month          700 non-null    int32  
 1   Region         700 non-null    object 
 2   Fertilizer     700 non-null    object 
 3   soil_moisture  700 non-null    float64
 4   Temperature    700 non-null    float64
 5   Rainfall       700 non-null    float64
 6   Humidity       700 non-null    float64
 7   ph_level       700 non-null    float64
dtypes: float64(5), int32(1), object(2)
memory usage: 46.5+ KB
None


In [64]:
#레이블인코딩
from sklearn.preprocessing import LabelEncoder

#범주형 변수 지정
categorical_col=['Region', 'Fertilizer']

#레이블 인코더 생성
label_encoder = LabelEncoder()

#범주형 컬럼에 대해 레이블 인코딩 적용
for col in categorical_col:
    x_train2[col]=label_encoder.fit_transform(x_train2[col])
    x_test[col]=label_encoder.transform(x_test2[col])

#인코딩 데이터 확인 
print(x_train2.info())
print(x_test2.info())
print(x_train2.head())

<class 'pandas.core.frame.DataFrame'>
Index: 700 entries, 521 to 150
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Month          700 non-null    int32  
 1   Region         700 non-null    int32  
 2   Fertilizer     700 non-null    int32  
 3   soil_moisture  700 non-null    float64
 4   Temperature    700 non-null    float64
 5   Rainfall       700 non-null    float64
 6   Humidity       700 non-null    float64
 7   ph_level       700 non-null    float64
dtypes: float64(5), int32(3)
memory usage: 41.0 KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 300 entries, 788 to 102
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Month          300 non-null    int32  
 1   Region         300 non-null    object 
 2   Fertilizer     300 non-null    object 
 3   soil_moisture  300 non-null    float64
 4   Temperature    300 non-null    fl

In [65]:
#데이터 분리 - 데이터를 훈련 세트와 검증용 세트로 분할 80%분할, 20%검증
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2, stratify=y_train, random_state=2025) #stratify=y_train 클래스가 불균형 옵션일 때 넣어준다

print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)


(560, 13)
(140, 13)
(560,)
(140,)


In [66]:
#모델링 및 성능평가
#랜덤포레스트 모델 사용 (참고: RandomForestRegressor)
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier(random_state=2025)
model.fit(x_train,y_train)

RandomForestClassifier(random_state=2025)

In [67]:
#모델을 사용하여 테스트 데이터 예측
y_pred = model.predict(x_val)

In [68]:
#모델 성능 평가(정확도, F1-Score, 민감도, 특이도)
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, recall_score, precision_score
acc = accuracy_score(y_val, y_pred)#실제값, 예측값
f1=f1_score(y_val, y_pred, average='macro')
#정확도는 불균형 데이터 특정 성능 지표가 아니다. 
print('Acc:', acc)
print('macro F1:', f1)

Acc: 0.7071428571428572
macro F1: 0.2773109243697479


In [70]:
# 예측 전에 컬럼을 학습 때와 동일하게 맞추기
x_test_aligned = x_test[x_train.columns]

# 예측
y_result = model.predict(x_test_aligned)

# 결과 저장
result = pd.DataFrame({'pred': y_result})
result.to_csv('result.csv', index=False)
print(result.head())


   pred
0     0
1     0
2     0
3     0
4     0
